# Churn Modelling(Müşteri Kayıp Analiz Modeli)

Kütüphanelerin import edilmesi

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Datanın okutulması

In [2]:
veriler=pd.read_csv('Churn.csv')

In [3]:
print(veriler)

      RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
0             1    15634602   Hargrave          619    France  Female   42   
1             2    15647311       Hill          608     Spain  Female   41   
2             3    15619304       Onio          502    France  Female   42   
3             4    15701354       Boni          699    France  Female   39   
4             5    15737888   Mitchell          850     Spain  Female   43   
...         ...         ...        ...          ...       ...     ...  ...   
9995       9996    15606229   Obijiaku          771    France    Male   39   
9996       9997    15569892  Johnstone          516    France    Male   35   
9997       9998    15584532        Liu          709    France  Female   36   
9998       9999    15682355  Sabbatini          772   Germany    Male   42   
9999      10000    15628319     Walker          792    France  Female   28   

      Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMemb

Veri Ön İşleme

İlk 3 değerin işimize yaramadığını düşündüğüm için onları almamayı tercih ediyorum.

In [4]:
X= veriler.iloc[:,3:13].values
Y=veriler.iloc[:,13].values

Coğrafya ve Cinsiyet kolonunu label encoder yaparak matrise dönüştürmeye çalışıyoruz.

In [5]:
from sklearn import preprocessing

In [6]:
l1=preprocessing.LabelEncoder()
X[:,1]=l1.fit_transform(X[:,1])

In [7]:
l2=preprocessing.LabelEncoder()
X[:,2]=l2.fit_transform(X[:,2])

Coğrafya kolonu ikiden fazla değişken içerdiği için one hot encoder dönüşümü yapıyoruz.

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [9]:
ohe=ColumnTransformer([('ohe', OneHotEncoder(dtype=float),[1])], remainder="passthrough")
x=ohe.fit_transform(X)
x=X[:,1:]

Eğitim ve test verisinin oluşturulması

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.33,random_state=0)

Veri ölçeklendirme

In [12]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [14]:
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)

Ölçeklendirilen verilerin boyutuna bakılması

In [15]:
x_train.shape

(6700, 10)

In [16]:
y_train.shape

(6700,)

Yapay Sinir Ağlarına(ANN) Giriş için kütüphanelerin import edilmesi

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense

ANN için giriş, gizli ve çıkış katmanının oluşturulması

In [20]:
model=Sequential()
model.add(Dense(6,activation='relu'))
model.add(Dense(6,activation='relu'))
model.add(Dense(6,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [35]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Model eğitiminin devir sayısının belirlenmesi

In [37]:
model.fit(x=x_train,y=y_train, epochs=50)

Epoch 1/50
210/210 [==============================] - 0s 744us/step - loss: 0.3263 - accuracy: 0.8660
Epoch 2/50
210/210 [==============================] - 0s 659us/step - loss: 0.3263 - accuracy: 0.8670
Epoch 3/50
210/210 [==============================] - 0s 691us/step - loss: 0.3262 - accuracy: 0.8663
Epoch 4/50
210/210 [==============================] - 0s 702us/step - loss: 0.3259 - accuracy: 0.86580s - loss: 0.3254 - accuracy: 0.86
Epoch 5/50
210/210 [==============================] - 0s 716us/step - loss: 0.3261 - accuracy: 0.8669
Epoch 6/50
210/210 [==============================] - 0s 709us/step - loss: 0.3256 - accuracy: 0.8672
Epoch 7/50
210/210 [==============================] - 0s 722us/step - loss: 0.3259 - accuracy: 0.8663
Epoch 8/50
210/210 [==============================] - 0s 662us/step - loss: 0.3261 - accuracy: 0.8645
Epoch 9/50
210/210 [==============================] - 0s 728us/step - loss: 0.3258 - accuracy: 0.86520s - loss: 0.3291 - accuracy: 0.
Epoch 10/50
210/

Bağımlı değişkenin tahmin edilmesi

In [42]:
y_pred=model.predict(x_test)

Aşağıda bulunan kod, değerler ondalıklı olduğu için 0-1 arasında almamızı sağlamaktadır.

In [43]:
y_pred=(y_pred>0.5)

Hata matrisinin(Confusion matrix) oluşturulması

In [44]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)

In [46]:
print(cm)

[[2523   94]
 [ 368  315]]


Modele göre 2523 kişi sitede kalmayı tercih edeceği, 315 kişi de siteden ayrılmayı tercih edeceği tahmin edilmiştir.